In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import sys
from scipy import sparse
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append("../utils/")
import preprocessing
import sci_excerpts
import anndata2ri
import rpy2.robjects as ro
import rpy2.rinterface_lib.callbacks

In [2]:
adata = sc.read_h5ad("../../process/pre-intergration/big_data/20241007_mergeall_filter_gene_step1.h5ad")

In [3]:
n_pcs=50
counts_per_cell = 1e4
louvain_r=0.5
log_transform=False
anndata2ri.activate()
ro.r('library("scran")')
    
    # keep raw counts
adata.layers["counts"] = adata.X.copy()
    
    # Preliminary clustering for differentiated normalisation
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=counts_per_cell)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=n_pcs, svd_solver='arpack')
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=louvain_r)  

ro.globalenv['data_mat'] = adata.X.T
ro.globalenv['input_groups'] = adata_pp.obs['groups']
size_factors = ro.r('sizeFactors(computeSumFactors(SingleCellExperiment('
                            'list(counts=data_mat)), clusters = input_groups,'
                            f' min.mean = {0.1}))')

R[write to console]: 载入需要的程辑包：SingleCellExperiment

R[write to console]: 载入需要的程辑包：SummarizedExperiment

R[write to console]: 载入需要的程辑包：MatrixGenerics

R[write to console]: 载入需要的程辑包：matrixStats

R[write to console]: 
载入程辑包：‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, 

RRuntimeError: Error in (function (cond)  : 
  在为'sizeFactors'函数选择方法时评估'object'参数出了错: BiocParallel errors
  1 remote errors, element index: 1
  148 unevaluated and other errors
  first remote error:
Error in as(x, "dgCMatrix"): 没有可以用来強制转换“dgRMatrix”成“dgCMatrix”的方法或默认函数
 


In [5]:
data_mat_type = ro.r('class(data_mat)')

In [8]:
ro.r('data_mat <- as(data_mat, "CsparseMatrix")')

<56053x372369 sparse matrix of type '<class 'numpy.float64'>'
	with 1051537667 stored elements in Compressed Sparse Column format>

In [ ]:
size_factors = ro.r('sizeFactors(computeSumFactors(SingleCellExperiment('
                            'list(counts=data_mat)), clusters = input_groups,'
                            f' min.mean = {0.1}))')

In [11]:
sizeTb = pd.DataFrame(size_factors)

In [13]:
!mkdir ../../process/pre-intergration/table

In [14]:
sizeTb.to_csv("../../process/pre-intergration/table/size_factor.csv")

In [3]:
sizeTb = pd.read_csv("../../process/pre-intergration/table/size_factor.csv",index_col=0)

In [5]:
sizeTb['0']

0         0.340327
1         0.198762
2         1.265106
3         0.233104
4         0.787304
            ...   
372364    0.884356
372365    0.810209
372366    0.628486
372367    0.985773
372368    0.856885
Name: 0, Length: 372369, dtype: float64

In [20]:
adata.obs['size_factors'] = sizeTb['0'].values

In [26]:
adata.X[1:10,1:10].toarray()

array([[0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0.]])

In [27]:
    # modify adata
adata.X /= adata.obs['size_factors'].values[:,None]

    # convert to sparse, bc operation always converts to dense
adata.X = sparse.csr_matrix(adata.X)

In [23]:
#adata.X = adata.layers["counts"]

In [10]:
adata.layers["counts"] = adata.X

In [28]:
adata.write_h5ad("../../process/pre-intergration/big_data/20241007_mergeall_normalized_step2.h5ad")

In [29]:
adata.X[0, :30].toarray() * adata.obs.size_factors[0]

/tmp/ipykernel_577685/1771627096.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adata.X[0, :30].toarray() * adata.obs.size_factors[0]


array([[0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [41]:
adata.obs["Sample"] = adata.obs['orig.ident'].copy()

In [43]:
!mkdir ../../processed_data

In [44]:
!mkdir ../../processed_data/metadata

In [45]:
adata.obs.to_csv("../../processed_data/metadata/20241008_metadata.csv")

In [34]:
gb_values = sns.color_palette("Set2", 26)
color_labels = adata.obs["orig.ident"].unique()
color_map = dict(zip(color_labels, gb_values))
sc.pp.calculate_qc_metrics(adata, inplace=True, layer="counts")

In [ ]:
plt.scatter(
    adata.obs.total_counts,
    adata.obs.size_factors,
    c=list(map(lambda x: color_map.get(x), adata.obs["orig.ident"].values)),
    s=1,
)
plt.xlabel("total counts")
plt.ylabel("size factor")
plt.show()